In [1]:

from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np 
import argparse
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential

def load_model():

    model = Sequential()
    
    # spatiotemporal Encoder layer
    model.add(Conv3D(filters = 128, kernel_size =(11,11,1), strides = (4,4,1), padding = 'valid', input_shape = (227,227,10,1), activation = 'tanh'))
    model.add(Conv3D(filters=64, kernel_size = (5,5,1), strides = (2,2,1), padding = 'valid', activation = 'tanh'))

    model.add(ConvLSTM2D(filters = 64, kernel_size = (3,3), strides = 1, padding = 'same', dropout = 0.4, recurrent_dropout = 0.3, return_sequences = True))
    model.add(ConvLSTM2D(filters = 32, kernel_size = (3,3), strides = 1, padding = 'same', dropout = 0.4, return_sequences = True))
    model.add(ConvLSTM2D(filters = 64, kernel_size = (3,3), strides = 1, padding = 'same', dropout = 0.5, return_sequences = True))

    # spatiotemporal Decoder layer
    model.add(Conv3DTranspose(filters = 128, kernel_size = (5,5,1), strides = (2,2,1), padding = 'valid', activation = 'tanh'))
    model.add(Conv3DTranspose(filters = 1, kernel_size = (11,11,1), strides = (4,4,1), padding = 'valid', activation = 'tanh'))

    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

    return model

X_train=np.load('trainer.npy')
frames=X_train.shape[2]
#Need to make number of frames divisible by 10 to ease the load_model


frames=frames-frames%10

X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()


epochs=200
batch_size=1



if __name__=="__main__":

	model=load_model()

	callback_save = ModelCheckpoint("AnomalyDetector.h5",
									monitor="mean_squared_error")

	callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

	print('Trainer has been loaded')
	model.fit(X_train,Y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  callbacks = [callback_save,callback_early_stopping]
			  )
    
print('Done\n Please wait while AnomalyDetector.h5 has been created \nRefresh when needed')

Trainer has been loaded
Epoch 1/200


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.